In [3]:
# ======================================================================
# ANALISIS ULANG KETERSEDIAAN AIR (WATER YIELD) 2016 (METODE BARU)
# ======================================================================

# --- Langkah 1 & 2: Setup, AOI, dan Presipitasi ---
import ee
import geemap
ee.Authenticate()
ee.Initialize()
Map = geemap.Map()

path_aset_lc_2016 = "users/ivan_taslim/LC_Paguyaman_2016_DW_Filled"
lc_2016 = ee.Image(path_aset_lc_2016)
aoi = lc_2016.geometry()
Map.addLayer(aoi, {'color': 'blue'}, 'Batas DAS Paguyaman (AOI)')
Map.centerObject(aoi, 11)

print("Info: Menggunakan data Presipitasi tahun 2016")
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
precipitation_2016 = chirps.filter(ee.Filter.date('2016-01-01', '2016-12-31')).sum().clip(aoi)

# --- Langkah 3: Menghitung Evapotranspirasi 2016 dari ERA5-Land ---
print("Info: Menggunakan data ET dari ERA5-Land untuk tahun 2016.")
era5_et = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR').filter(ee.Filter.date('2016-01-01', '2016-12-31'))

# Pilih band evapotranspirasi total dan jumlahkan untuk setahun
# Nilai asli dalam meter, kita konversi ke milimeter
total_et_band = era5_et.select('total_evaporation_sum')
et_annual_sum_2016 = total_et_band.sum().multiply(1000).clip(aoi)


# --- Langkah 4: Menghitung Water Yield 2016 ---
print("Menghitung Water Yield dan Volume Total...")
water_yield_2016 = precipitation_2016.subtract(et_annual_sum_2016).rename('water_yield')
aoi_area_m2 = aoi.area().getInfo()
mean_yield_mm = water_yield_2016.reduceRegion(
    reducer=ee.Reducer.mean(), geometry=aoi, scale=30, maxPixels=1e12
).get('water_yield').getInfo()
mean_yield_m = mean_yield_mm / 1000
total_water_yield_volume_2016 = aoi_area_m2 * mean_yield_m

# Menampilkan hasil
print("======================================================================")
print("HASIL ANALISIS ULANG KETERSEDIAAN AIR (WATER YIELD) TAHUN 2016")
print("----------------------------------------------------------------------")
print(f"Rata-rata Water Yield: {mean_yield_mm:.2f} mm/tahun")
print(f"Total Volume Air Tersedia: {total_water_yield_volume_2016:,.2f} meter kubik/tahun")
print("======================================================================")

Map

Info: Menggunakan data Presipitasi tahun 2016
Info: Menggunakan data ET dari ERA5-Land untuk tahun 2016.
Menghitung Water Yield dan Volume Total...
HASIL ANALISIS ULANG KETERSEDIAAN AIR (WATER YIELD) TAHUN 2016
----------------------------------------------------------------------
Rata-rata Water Yield: 3319.15 mm/tahun
Total Volume Air Tersedia: 12,553,798,792.35 meter kubik/tahun


Map(center=[0.7139019338201237, 122.37991662440572], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
# ======================================================================
# EKSPOR PETA WATER YIELD 2016
# ======================================================================

# Ekspor ke GEE Assets
task_asset = ee.batch.Export.image.toAsset(
  image=water_yield_2016.toFloat(), # Konversi ke float untuk ekspor
  description='WaterYield_Paguyaman_2016Fixed_to_Asset',
  assetId='users/ivan_taslim/WaterYield_Paguyaman_2016Fixed', # Path aset tujuan Anda
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_asset.start()
print("Proses ekspor ke GEE ASSETS telah dimulai.")

# Ekspor ke Google Drive
task_drive = ee.batch.Export.image.toDrive(
  image=water_yield_2016.toFloat(),
  description='WaterYield_Paguyaman_2016Fixed_to_Drive',
  folder='Hasil_Analisis_DAS_Paguyaman',
  fileNamePrefix='WaterYield_Paguyaman_2016Fixed',
  region=aoi,
  scale=30,
  maxPixels=1e12
)
task_drive.start()
print("Proses ekspor ke GOOGLE DRIVE telah dimulai.")
print("\nPERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.")

Proses ekspor ke GEE ASSETS telah dimulai.
Proses ekspor ke GOOGLE DRIVE telah dimulai.

PERIKSA TAB 'TASKS' DI GEE CODE EDITOR UNTUK MENJALANKAN KEDUA PROSES EKSPOR.
